Read the data in the text file.

In [1]:
file = open('tennis_data.txt','r')
content = file.read()
print(content)

D1,Sunny,Hot,High,Weak,No
D2,Sunny,Hot,High,Strong,No
D3,Overcast,Hot,High,Weak,Yes
D4,Rain,Mild,High,Weak,Yes
D5,Rain,Cool,Normal,Weak,Yes
D6,Rain,Cool,Normal,Strong,No
D7,Overcast,Cool,Normal,Strong,Yes
D8,Sunny,Mild,High,Weak,No
D9,Sunny,Cool,Normal,Weak,Yes
D10,Rain,Mild,Normal,Weak,Yes
D11,Sunny,Mild,Normal,Strong,Yes
D12,Overcast,Mild,High,Strong,Yes
D13,Overcast,Hot,Normal,Weak,Yes
D14,Rain,Mild,High,Strong,No


Converting the text file to csv

In [5]:
import pandas as pd
#store the text file in a dataframe
df = pd.read_csv("tennis_data.txt")
#convert the dataframe to csv
df.to_csv('tennis_data.csv', index=None)

with open('tennis_data.csv', 'r') as file:
    print(file.read(1000))

D1,Sunny,Hot,High,Weak,No
D2,Sunny,Hot,High,Strong,No
D3,Overcast,Hot,High,Weak,Yes
D4,Rain,Mild,High,Weak,Yes
D5,Rain,Cool,Normal,Weak,Yes
D6,Rain,Cool,Normal,Strong,No
D7,Overcast,Cool,Normal,Strong,Yes
D8,Sunny,Mild,High,Weak,No
D9,Sunny,Cool,Normal,Weak,Yes
D10,Rain,Mild,Normal,Weak,Yes
D11,Sunny,Mild,Normal,Strong,Yes
D12,Overcast,Mild,High,Strong,Yes
D13,Overcast,Hot,Normal,Weak,Yes
D14,Rain,Mild,High,Strong,No



Add headers to my csv file and print the dataframe.

In [6]:
from io import StringIO
# Add headers to columns
headers = ['Day',	'Outlook',	'Temperature',	'Humidity',	'Wind',	'PlayTennis']
    
# Read CSV string into DataFrame
df = pd.read_csv(StringIO(content), header=None, names=headers)

# Display the DataFrame
print(df)

    Day   Outlook Temperature Humidity    Wind PlayTennis
0    D1     Sunny         Hot     High    Weak         No
1    D2     Sunny         Hot     High  Strong         No
2    D3  Overcast         Hot     High    Weak        Yes
3    D4      Rain        Mild     High    Weak        Yes
4    D5      Rain        Cool   Normal    Weak        Yes
5    D6      Rain        Cool   Normal  Strong         No
6    D7  Overcast        Cool   Normal  Strong        Yes
7    D8     Sunny        Mild     High    Weak         No
8    D9     Sunny        Cool   Normal    Weak        Yes
9   D10      Rain        Mild   Normal    Weak        Yes
10  D11     Sunny        Mild   Normal  Strong        Yes
11  D12  Overcast        Mild     High  Strong        Yes
12  D13  Overcast         Hot   Normal    Weak        Yes
13  D14      Rain        Mild     High  Strong         No


Create conditional probability tables (CPTs) for each of the random variables.

In [51]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

def single_column_cpt(series):
    """Create probability table from single column"""
    counts = series.value_counts()
    probabilities = series.value_counts(normalize=True)
    
    cpt = pd.DataFrame({
        'Count': counts,
        'Probability': probabilities
    })
    
    return cpt

cpt = single_column_cpt(df['PlayTennis'])
print("CPT for Play Tennis:")
print(cpt)
print()

def create_cpt(df,target_col, condition_cols):
    """
    Create CPT for P(target | conditions)
    """
    grouped = df.groupby(target_col)[condition_cols].value_counts(normalize=True)
    return grouped.unstack(fill_value=0)

# output for the data
outlook = create_cpt(df, 'PlayTennis', ['Outlook'])
print(outlook)
print()
temp = create_cpt(df, 'PlayTennis', ['Temperature'])
print(temp)
print()
humidity = create_cpt(df, 'PlayTennis', ['Humidity'])
print(humidity)
print()
wind = create_cpt(df, 'PlayTennis', ['Wind'])
print(wind)
print()



CPT for Play Tennis:
            Count  Probability
PlayTennis                    
Yes             9     0.642857
No              5     0.357143

Outlook     Overcast      Rain     Sunny
PlayTennis                              
No          0.000000  0.400000  0.600000
Yes         0.444444  0.333333  0.222222

Temperature      Cool       Hot      Mild
PlayTennis                               
No           0.200000  0.400000  0.400000
Yes          0.333333  0.222222  0.444444

Humidity        High    Normal
PlayTennis                    
No          0.800000  0.200000
Yes         0.333333  0.666667

Wind          Strong      Weak
PlayTennis                    
No          0.600000  0.400000
Yes         0.333333  0.666667



Cpt from a single column

In [49]:
def single_column_cpt(series):
    """Create probability table from single column"""
    counts = series.value_counts()
    probabilities = series.value_counts(normalize=True)
    
    cpt = pd.DataFrame({
        'Count': counts,
        'Probability': probabilities
    })
    
    return cpt

cpt = single_column_cpt(df['PlayTennis'])
print("CPT for Play Tennis:")
print(cpt)

CPT for Play Tennis:
            Count  Probability
PlayTennis                    
Yes             9     0.642857
No              5     0.357143


Carrying out probability tests from the data in the cpt tables.

In [ ]:
wind_temp = create_cpt(df, 'PlayTennis', ['Wind', 'Temperature'])
print(wind_temp)
print()
'''
#This queries data in dictionary format
def query_cpt(cpt_data, conditions, target_value):
    """Query CPT for specific probability"""
    if conditions in cpt_data:
        return cpt_data[conditions].get(target_value, 0)
    return 0

# Test: P(Yes | strong, mild)
prob = query_cpt(wind_temp, ('Strong', 'Mild'), 'Yes')
print(f"P(Yes | strong, mild) = {prob}")
'''
#Query data in a dataframe format using .loc
def query_cpt(cpt_df, wind_val, temp_val, target_value):
    """Query CPT DataFrame for specific probability"""
    try:
        return cpt_df.loc[(target_value, wind_val), temp_val]
    except (KeyError, IndexError):
        return 0.0

# Test: P(Yes | Strong, Mild)
prob = query_cpt(wind_temp, 'Strong', 'Mild', 'Yes')
print(f"P(Yes | Strong, Mild) = {prob}")

Temperature            Cool       Hot      Mild
PlayTennis Wind                                
No         Strong  0.200000  0.200000  0.200000
           Weak    0.000000  0.200000  0.200000
Yes        Strong  0.111111  0.000000  0.222222
           Weak    0.222222  0.222222  0.222222

P(Yes | Strong, Mild) = 0.2222222222222222


In [41]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define network structure
model = BayesianNetwork([('Humidity', 'PlayTennis'), ('Wind', 'PlayTennis')])

# Define CPDs
cpd_wind = TabularCPD(variable='Wind', variable_card=2, 
                     values=[[0.6, 0.33], #strong
                             [0.4, 0.67]],#weak

                     state_names={'Wind': ['Strong', 'Weak']})

cpd_humidity = TabularCPD(variable='Humidity', variable_card=2, 
                          values=[[0.33, 0.8],#High
                                  [0.67, 0.2]], #Normal
                          state_names={'Humidity': ['High', 'Normal']})

cpd_playtennis = TabularCPD(variable='PlayTennis', variable_card=2,
                         values=[[0.64],[0.36]],  
                        
                         evidence=['Wind', 'Humidity'],
                         evidence_card=[2, 2],
                         state_names={'PlayTennis': ['Yes', 'No'],
                                      'Wind': ['Strong', 'Weak'],
                                      'Humidity': ['High', 'Normal']})

# Add CPDs to model
model.add_cpds(cpd_wind, cpd_humidity, cpd_playtennis)

# Verify model
print("Model is valid:", model.check_model())

# Perform inference
inference = VariableElimination(model)

# Query: P(Disease | Symptoms=fever, Age=young)
result = inference.query(variables=['PlayTennis'], 
                        evidence={'Humidity': 'Normal', 'Wind': 'Weak'})
print("P(PlayTennis | Normal, Weak):")
print(result)

ImportError: cannot import name '_lazywhere' from 'scipy._lib._util' (c:\Users\THINKPAD\OneDrive - hull.ac.uk\GitHub\bayes_theorem\.venv\Lib\site-packages\scipy\_lib\_util.py)